In [7]:
import os
import logging
from dotenv import load_dotenv
from pymilvus import connections, Collection
from ibm_watsonx_ai import Credentials
from ibm_watsonx_ai.foundation_models import Embeddings, ModelInference
from ibm_watsonx_ai.foundation_models.utils.enums import EmbeddingTypes, ModelTypes

In [8]:
# Load .env values
load_dotenv(override=True)
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

COS_API_KEY = os.getenv("COS_API_KEY")
WATSON_STUDIO_PROJECT_ID = os.getenv("WATSON_STUDIO_PROJECT_ID")
MILVUS_HOST = os.getenv("MILVUS_GRPC_HOST")
MILVUS_PORT = int(os.getenv("MILVUS_GRPC_PORT"))
MILVUS_CERT_PATH = os.getenv("MILVUS_CERT_PATH")



In [9]:
# ─── Watsonx Credentials ────────────────────────────────────────────────

logging.info("Initializing Watsonx credentials")
client = Credentials(url="https://us-south.ml.cloud.ibm.com", api_key=COS_API_KEY)

logging.info("Loading embedding model")
embedding = Embeddings(
    model_id=EmbeddingTypes.IBM_SLATE_30M_ENG,
    project_id=WATSON_STUDIO_PROJECT_ID,
    credentials=client
)

logging.info("Loading LLM (ModelInference)")
llm = ModelInference(
    model_id=ModelTypes.GRANITE_13B_INSTRUCT_V2,   # ibm/granite-13b-instruct-v2
    project_id=WATSON_STUDIO_PROJECT_ID,
    credentials=client
)


2025-03-30 16:36:00,610 - INFO - Initializing Watsonx credentials
2025-03-30 16:36:00,611 - INFO - Loading embedding model
2025-03-30 16:36:01,013 - INFO - Client successfully initialized
2025-03-30 16:36:01,192 - INFO - Loading LLM (ModelInference)
2025-03-30 16:36:01,791 - INFO - Client successfully initialized
2025-03-30 16:36:03,155 - INFO - HTTP Request: GET https://us-south.ml.cloud.ibm.com/ml/v1/foundation_model_specs?version=2025-03-20&project_id=cd99c1e5-2311-4803-b050-680fe88d1a77&filters=function_text_generation%2C%21lifecycle_withdrawn%3Aand&limit=200 "HTTP/1.1 200 OK"
2025-03-30 16:36:03,199 - INFO - Successfully finished Get available foundation models for url: 'https://us-south.ml.cloud.ibm.com/ml/v1/foundation_model_specs?version=2025-03-20&project_id=cd99c1e5-2311-4803-b050-680fe88d1a77&filters=function_text_generation%2C%21lifecycle_withdrawn%3Aand&limit=200'


In [10]:
# ─── Connect to Milvus ──────────────────────────────────────────────────

connections.connect(
    alias="default",
    host=MILVUS_HOST,
    port=MILVUS_PORT,
    user="ibmlhapikey",
    password=COS_API_KEY,
    secure=True,
    server_ca=MILVUS_CERT_PATH
)

In [11]:
# ─── Generation Function ────────────────────────────────────────────────

def generate_answer(query: str, collection_name: str = "carbon_embeddings", top_k: int = 5) -> str:
    logging.info(f"Generating answer for query: {query}")

    # Step 1: Embed the query
    query_vector = embedding.embed_documents([query])[0]

    # Step 2: Search Milvus
    collection = Collection(name=collection_name)
    collection.load()
    search_params = {"metric_type": "L2", "params": {"nprobe": 10}}

    results = collection.search(
        data=[query_vector],
        anns_field="embedding",
        param=search_params,
        limit=top_k,
        output_fields=["chunk_text", "file_name"]
    )

    # Step 3: Gather context
    context_chunks = []
    for hit in results[0]:
        chunk = hit.entity.get("chunk_text", "")
        context_chunks.append(chunk.strip())
    context = "\n".join(context_chunks)

    logging.info(f"Retrieved {len(context_chunks)} relevant chunks from Milvus.")

    # Step 4: Construct prompt
    prompt = f"""Use the context below to answer the user's question as clearly and accurately as possible.

Context:
{context}

Question:
{query}

Answer:"""

    # Step 5: Generate response from Watsonx LLM
    response = llm.generate(prompt=prompt)
    answer = response['results'][0]['generated_text']
    return answer


In [12]:
print("💬 CarbonSense | Ask your sustainability question")

try:
    query = "Soyabean".strip()
    if query.lower() == "exit":
        pass

    answer = generate_answer(query)
    print("\n🧠 Answer:\n", answer)

except Exception as e:
    logging.error(f"Error during generation: {e}")


2025-03-30 16:36:03,483 - INFO - Generating answer for query: Soyabean


💬 CarbonSense | Ask your sustainability question


2025-03-30 16:36:04,708 - INFO - HTTP Request: POST https://us-south.ml.cloud.ibm.com/ml/v1/text/embeddings?version=2025-03-20 "HTTP/1.1 200 OK"
2025-03-30 16:36:04,710 - INFO - Successfully finished generate for url: 'https://us-south.ml.cloud.ibm.com/ml/v1/text/embeddings?version=2025-03-20'
2025-03-30 16:36:04,963 - ERROR - Error during generation: Hit.get() takes 2 positional arguments but 3 were given
